# Executive Summary

The goal of this challenge was to code an optimised search engine for recipes achieving bespoke results (/i.e. supporting the user's food preferences) with minimal wall time. A data set is provided. The search engine is to be pretty basic, returning all recipes that contain all of the provided keywords. However, the user can choose from a number of orderings depending on their food preferences, which the search engine should support.

## Specification

The system provides a function ``search``, with the following specification:
```
def search(query, ordering = 'normal', count = 10):
  ...
```

It `print`s out the results of the search, subject to the following rules:
1. It selects from the set of all recipes that contain __all__ of the words in the query (the positions/order of the words in the recipe are to be ignored).
2. It orders them based on the provided ordering (a string, meaning defined below).
3. It `print`s the top `count` matches only, preserving the order from best to worst. The search engine `print`s just their title, one per line.


### Data set

A file, `recipes.json` is provided, containing 17K recipes. It can be parsed into a Python data structure using the [`json`](https://docs.python.org/3/library/json.html) module. It is a list, where each recipe is a dictionary containing various keys:
* `title` : Name of recipe; you can assume these are unique
* `categories` : A list of tags assigned to the recipe
* `ingredients` : What is in it, as a list
* `directions` : List of steps to make the recipe
* `rating` : A rating, out of 5, of how good it is
* `calories` : How many calories it has
* `protein` : How much protein is in it
* `fat` : How much fat is in it

Note that the data set was obtained via web scrapping and hence is noisy - every key except for `title` is missing from at least one recipe. The code will need to cope with this.

The data set comes from https://www.kaggle.com/hugodarwood/epirecipes/version/2 though it has undergone some preprocessing/cleaning, i.e. removing of duplicates and really 'dodgy' entries.



### Search

The search should check the following parts of the recipe (see data set description below):
* `title`
* `categories`
* `ingredients`
* `directions`

For instance, given the query "banana cheese" you would expect "Banana Layer Cake with Cream Cheese Frosting" in the results. Note that case is ignored ("banana" matches "Banana") and the words __do not__ have to be next to one another, in the same order as the search query or even in the same part of the recipe ("cheese" could appear in the title and "banana" in the ingredients). However, all words in the search query __must__ appear somewhere.



### Tokenisation

In order to match words, tokenisation is carried out as follows:
1. All punctuation and digits are converted into spaces. For punctuation I use the set in [`string.punctuation`](https://docs.python.org/3/library/string.html#string.punctuation), for digits [`string.digits`](https://docs.python.org/3/library/string.html#string.digits).
2. [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) to extract individual tokens.
3. Ignore any token that is less than $3$ characters long.
4. Make tokens lowercase.

When matching words for search it's only a match if the entire token is matched (There are many scenarios where this simple approach will fail, but it's good enough for this exercise). When carrying out a search the code should ignore terms in the search string that fail the above requirements.



### Ordering

There are three ordering modes to select from / that the search engine will offer, each indicated by passing a string to the `search` function:
* `normal` - Based simply on the number of times the search terms appear in the recipe. A score is calculated and the order is highest to lowest. The score sums the following terms (repeated words are counted multiple times, i.e. "cheese cheese cheese" is $3$ matches to "cheese"):
    * $8 \times$ Number of times a query word appears in the title
    * $4 \times$ Number of times a query word appears in the categories
    * $2 \times$ Number of times a query word appears in the ingredients
    * $1 \times$ Number of times a query word appears in the directions
    * The `rating` of the recipe (if not available assume $0$).

* `simple` - Tries to minimise the complexity of the recipe, for someone who is in a rush. Orders to minimise the number of ingredients multiplied by the numbers of steps in the directions.

* `healthy` - Order from lowest to highest by this cost function:
$$\frac{|\texttt{calories} - 510n|}{510} + 2\frac{|\texttt{protein} - 18n|}{18} + 4\frac{|\texttt{fat} - 150n|}{150}$$
Where $n \in \mathbb{N}^+$ is selected to minimise the cost ($n$ is a positive integer and $n=0$ is not allowed). This can be understood in terms of the numbers $510$, $18$ and $150$ being a third of the recommended daily intake (three meals per day) for an average person, and $n$ being the number of whole meals the person gets out of cooking/making the recipe. So this tries to select recipes that neatly divide into a set of meals that are the right amount to consume for a healthy, balanced diet.

To clarify the use of the ordering string, to get something healthy that contains cheese one might call `search('cheese', 'healthy')`. 

In the case of a recipe that is missing a key in its dictionary the rules are different for each search mode:
* `normal` - Consider a missing entry to be zero matches, but the item is still eligible for inclusion in the results, assuming it can match the search with a different entry.
* `simple` - If a recipe is missing either `ingredients` or `directions` it is dropped from such a search result. Because the data is messy if either of these lists is of length $1$ it should be assumed that the list extraction has failed and the recipe is to also be dropped from the search results.
* `healthy` - If any of `calories`, `protein` or `fat` is missing the recipe should be dropped from the result.


(Finally, goal is to achieve a search that it's faster than $0.1$ seconds on average)

### Preprocessing

In [1]:
#Import and read json file
#import time
import json
import numpy as np

with open('recipes.json') as recipes:
    json_file = json.load(recipes)

In [2]:
#Create translate table
import string

#Function tokenize_process:
#1) gets rid of numbers and punctuation marks and replace with empty string
#2) splits strings into their individual words/tokens
#3) remove all strings less than 3 characters
#4) make everything lowercase

def tokenize_process(strings):
    #Here we create the translation table 
    translation = str.maketrans(string.punctuation + string.digits, ' '*len(string.punctuation+string.digits))
    
    strings_ = str(strings) 
    translated = strings_.translate(translation) #1) Translates each punctuation sign or digit into an empty string
    trans_split = translated.split() #2) Splits up the string into several strings of individual tokens
    
    output = []
    #3) Here we remove all strings less than 3 characters:
    for i in trans_split:
        if len(i) >= 3:
            output.append(i.lower()) #4) here we append i as a lower case
    
    return output

In [3]:
#Create a list too loop through title categories ingredients and directions
look_thru_keys = list(json_file[0].keys())
look_thru_keys = look_thru_keys[:-1]

In [4]:
#Create dictionary recipe map which has tokens as keys and a list of values, which are indexes of the recipes
#Also for each token it checks whether it is in the title, categories, ingredients or directions and adds the
#index number several times in accordances with the order rank
recipe_map = {}
recipe_simple_score = {k : 0 for k in range(len(json_file))} #Here we create an empty scoredictionary for the 
                                                             #recipes for which we fill with relevant simple score
                                                             #in the loop
for keys in look_thru_keys:
    for i in range(len(json_file)):
        try:
            tokens = tokenize_process(json_file[i][keys])
            if len(json_file[i]['directions']) > 1 and len(json_file[i]['ingredients']) > 1:
                recipe_simple_score[i] = len(json_file[i]['directions']) * len(json_file[i]['ingredients'])
        except KeyError:
            continue
        for j in tokens:
            if j in recipe_map.keys():
                if keys == 'title':
                    for order in range(8):
                        recipe_map[j].append(i)
                if keys == 'categories':
                    for order in range(4):
                        recipe_map[j].append(i)
                if keys == 'ingredients':
                    for order in range(2):
                        recipe_map[j].append(i)
                if keys == 'directions':
                    recipe_map[j].append(i)
                        
            else:
                if keys == 'title':
                    recipe_map[j] = [i] * 8
                if keys == 'categories':
                    recipe_map[j] = [i] * 4
                if keys == 'ingredients':
                    recipe_map[j] = [i] * 2
                if keys == 'directions':
                    recipe_map[j] = [i]
                    
scores = np.array(list(recipe_simple_score.values())) #Turned scores into array

In [5]:
#Healthy scoring
nutrition = ['calories','protein','fat']
health_score = np.empty(len(json_file))

for h in range(len(json_file)):
    h_s = []
    if nutrition[0] in json_file[h] and nutrition[1] in json_file[h] and nutrition[2] in json_file[h]:
        for n in range(1,11):  
            sc = (np.fabs(json_file[h][nutrition[0]]-510*n)/510) + (2*(np.fabs(json_file[h][nutrition[1]]-18*n)/18)) +(4*(np.fabs(json_file[h][nutrition[2]]-150*n)/150))
            h_s.append(sc)
            min_sc = min(h_s)
        health_score[h] = min_sc
        
    else: 
        health_score[h] = 0

In [6]:
#Here we create an empty matrix with dimensions no. of rows = recipe_map length i.e. number of tokens
#and no. of columns = the no. of recipes
token_matrix = np.zeros((len(recipe_map),len(json_file)))

In [7]:
#Here we loop through the tokens/keys in the recipe_map dictionary and define the column index of the matrix 
#to signify a recipe number and the count of how many times that token appears in the given recipe 
#as the element in the matrix
j = 0
for k, i in recipe_map.items():
    idx, cnt = np.unique(i, return_counts=True)
    token_matrix[j][idx] = cnt
    j += 1

In [8]:
#ratings array:
ratings_array = np.empty(len(json_file))
for l in range(len(json_file)):
    rating = json_file[l]['rating'] if 'rating' in json_file[l] else 0
    ratings_array[l] = rating

In [9]:
query_idx = {k : i for i, k in enumerate(recipe_map)}

### Search Engine

In [10]:
def search(query, ordering = 'normal', count = 10):
    tokens_ = tokenize_process(query) #Tokenize query such that to match tokens already existing in json_file.
    try: 
        rows = [query_idx[k] for k in tokens_] #Find relevant rows in token matrix associated with the query
        if ordering == 'normal':
            columns = np.where((token_matrix[rows]>0).all(0)) #Slice all columns such that only intersects are handled
            intersect_counts = token_matrix[rows].sum(axis = 0)[columns] + ratings_array[columns] 
            
            #Sum the counts of the tokens/ matrix elements of the intersecting recipes to get count order
            
            cols = columns[0]
            scores_r = intersect_counts[intersect_counts.argsort()[::-1]] 

            recipes_to_ret = cols[intersect_counts.argsort()[::-1]][0:count] #Re-order recipe array to follow normal order
            
            search_res = []
            
            for i in recipes_to_ret:
                search_res.append(json_file[i]['title'])
                



        if ordering == 'simple':
            columns_1 = np.where((token_matrix[rows]>0).all(0)) #Columns/recipes relevant to query

            relevant_sc_1 = scores[columns_1] #Scores corresponding to relevant recipes
            bool_ = np.where(relevant_sc_1>0)

            columns_2 = columns_1[0][bool_[0]]
            relevant_sc_2 = relevant_sc_1[bool_[0]]

            recipes_to_ret_s = columns_2[relevant_sc_2.argsort()][0:count]
            
            search_res = []
            
            for i in recipes_to_ret_s:
                search_res.append(json_file[i]['title'])

        if ordering == 'healthy':
            columns_2 = np.where((token_matrix[rows]>0).all(0))
            relevant_sc_2 = health_score[columns_2]
            bool_2 = np.where(relevant_sc_2!=0)
            columns_3 = columns_2[0][bool_2[0]]

            relevant_sc_3 = relevant_sc_2[bool_2[0]]
            relevant_sc_3

            recipes_to_ret_h = columns_3[relevant_sc_3.argsort()][0:count]
            
            search_res = []
            
            for i in recipes_to_ret_h:
                search_res.append(json_file[i]['title'])
                
        return search_res
                
    except KeyError:
        pass
    

In [11]:
%timeit search('banana cheese cake', 'normal')

117 µs ± 2.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [12]:
results = search('banana cheese cake', 'normal')
for i in results:
    print(i, '\n')

Banana Layer Cake with Cream Cheese Frosting  

Banana Layer Cake with White Chocolate-Cream Cheese Frosting and Walnuts  

Fresh Banana Layer Cake  

Banana Coconut Crunch Cake  

Banana-Pineapple Layer Cake with Cream Cheese Frosting  

Mango-Banana Cake  

Banana Layer Cake  

Persimmon Cake with Cream Cheese Icing  

Carrot-Banana Cake  

Banana Cake with Sour Cream Frosting  

